# Tunable

## Introduction

`Tunable` is a class specifically designed to represent a tuning
problem. This class is a collection of `Hyperparameters` objects
and interacts with them as one. This `Tunable` is used by the `Tuners` in order to solve the tuning problem that this represents.

### Hyperparameters

BTB comes with the following `hyperparameters`:

- `BooleanHyperParam`: create a search space for boolean values i.e: `True` or `False`.
- `CategoricalHyperParam`: create a search space for categorical values i.e: "foo", "bar".
- `FloatHyperParam`: create a search space for `float` values i.e: `0.0 - 1.0`
- `IntHyperParam`: create a search space for `int` values i.e: `0 - 1`

The BTB hyperparameters are using two `spaces`:
- Hyperparameter space: the set of all values that are searched.
- Search Space: normalized search space $[0, 1]^K$.

There are the following methods to interact with the `hyperparameter`:

- `inverse_transform`: invert one or more search space values.
- `transform`: transform one or more hyperparameter values.
- `sample`: generate an array of `n_samples` random samples in the search space.

See an example below:

In [1]:
from btb.tuning.hyperparams import IntHyperParam

ihp = IntHyperParam(min=0, max=4)

We will sample one value of this hyperparameter: 

In [2]:
sample = ihp.sample(1)
sample

array([[0.3]])

As you can see, our sample is within the space of $[0, 1]$, let's now see what value is holding this `sample` by calling the method `inverse_transform`.

In [3]:
inverse = ihp.inverse_transform(sample)
inverse

array([[1]])

As you can see, now our value is within the `hyperparameter space`, wich is: $[0, 4]$.

Let's transform the value `4` to see its representation in the `search space`.

In [4]:
ihp.transform(4)

array([[0.9]])

As you can see, the representation of this number is `0.9` 

### Creating a Tunable

As we mentioned before, the `Tunable` works with a given collection of hyperparameters as one, which means, that our `Tunable` class is capable of calling the appropiate method for each hyperparameter. For example when calling `Tunable.sample` we will generate random samples for the collection of hyperparameters that is inside.

There are two ways to create a tunable:

- From a dictionary with the name of the parameter and its instance of the hyperparameter as value.
- From a dictionary representing the hyperparameters.

#### From a dictionary with the instance of the hyperparameters

We will start by importing the `hyperparameters` and the `Tunable`, creating a dictionary with the name of the
parameter and then creating the `Tunable` object:

In [5]:
from btb.tuning.hyperparams import (BooleanHyperParam, CategoricalHyperParam,
                                    FloatHyperParam, IntHyperParam)
from btb.tuning.tunable import Tunable

In [6]:
hyperparams = {
    'boolean_hp': BooleanHyperParam(),
    'categorical_hp': CategoricalHyperParam(['foo', 'bar']),
    'float_hp': FloatHyperParam(min=0, max=1),
    'int_hp': IntHyperParam(min=1, max=10),
}

As you can see, we have created a hyperparameter for each of the types available at the moment.
Now, let's create a `Tunable` instance and let's try it out.

In [7]:
tunable = Tunable(hyperparams)

#### From a dictionary representing the hyperparameters

The class `Tunable` allows you to instantiate `from_dict` wich accepts as an input a python dictionary containing as `key` the given name for the hyperparameter and as value a dictionary containing the following keys:

- `type` (str): ``bool`` for ``BoolHyperParam``, ``int`` for ``IntHyperParam``, ``float`` for ``FloatHyperParam``, ``str`` for ``CategoricalHyperParam``.

- `range` or `values` (list): range / values that this hyperparameter can take, in case of ``CategoricalHyperParam`` those will be used as the ``choices``, for ``NumericalHyperParams`` the ``min`` value will be used as the minimum value and the ``max`` value will be used as the ``maximum`` value.

- `default` (str, bool, int, float or None): The default value for the hyperparameter. 

The above `hyperparams` can be represented as follows:

In [8]:
hyperparams = {
    'boolean_hp': {
        'type': 'bool',
    },
    'categorical_hp': {
        'type': 'str',
        'values': ['foo', 'bar'],
        'default': 'foo'
    },
    'float_hp': {
        'type': 'float',
        'values': [0, 1]
    },
    'int_hp': {
        'type': 'int',
        'values': [1, 10],
    }
}

In [9]:
tunable = Tunable.from_dict(hyperparams)

Now, when we call the `sample` method of `tunable` we will create a sample for each one of the hyperparameters taht we specified above.

In [10]:
sample = tunable.sample(1)
sample

array([[1.        , 1.        , 0.        , 0.58846476, 0.15      ]])

As you can see, now we have more values than before. Basicly, our tuner, called each hyperparameter's sample and gave them to us as one.

We can use the `inverse_transform` method to read properly the values that were generated by the `sample`:

In [11]:
inverse = tunable.inverse_transform(sample)
inverse

,boolean_hp,categorical_hp,float_hp,int_hp
0,True,foo,0.588465,2


This returns a `pandas.DataFrame`, so we can use `.to_dict(orient='records')` if we would like to obtain a dictionary and use it as `kwargs`. 

In [12]:
inverse_dict = inverse.to_dict(orient='records')[0]
inverse_dict

{'boolean_hp': True,
 'categorical_hp': 'foo',
 'float_hp': 0.5884647611762291,
 'int_hp': 2}

Finally, we can use the `transform` method to conver the given values in to a `search space` again. 

In [13]:
tunable.transform(inverse_dict)

array([[1.        , 1.        , 0.        , 0.58846476, 0.15      ]])